In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import graphviz
from graphviz import Graph

import warnings
warnings.filterwarnings("ignore")

import acquire
import perpare

In [2]:
# Acquire Step
df = acquire.get_titanic_data()
df.head(2)

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0


In [3]:
train, validate, test = perpare.prep_titanic(df)
                          

In [4]:
train.head()


,survived,pclass,age,sibsp,parch,fare,alone,Q,S,male
583,0,1,36.000000,0,0,40.1250,1,0,0,1
337,1,1,41.000000,0,0,134.5000,1,0,0,0
50,0,3,7.000000,4,1,39.6875,0,0,1,1
218,1,1,32.000000,0,0,76.2917,1,0,0,0
31,1,1,29.916875,1,0,146.5208,0,0,0,0


In [5]:
X_train = train.drop(columns=["survived"])
y_train = train.survived

X_validate = validate.drop(columns=["survived"])
y_validate = validate.survived

X_test = test.drop(columns=["survived"])
y_test = test.survived

In [6]:
# Make the model
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

# Fit the model (on train and only train)
knn.fit(X_train, y_train)

# Use the model
# We'll evaluate the model's performance on train, first
y_predictions = knn.predict(X_train)

# Produce the classification report on the actual y values and this model's predicted y values
report = classification_report(y_train, y_predictions, output_dict=True)
print("KNN 5 neighbors")
pd.DataFrame(report)

KNN 5 neighbors


,0,1,accuracy,macro avg,weighted avg
precision,0.807571,0.716667,0.774648,0.762119,0.772819
recall,0.833876,0.678947,0.774648,0.756412,0.774648
f1-score,0.820513,0.697297,0.774648,0.758905,0.773408
support,307.000000,190.000000,0.774648,497.000000,497.000000


### LETS MAKE A LOOP

In [7]:
metrics = []

for i in range(1, 25 ):
    # Make the model
    knn = KNeighborsClassifier(n_neighbors=i, weights='uniform')

    # Fit the model (on train and only train)
    knn.fit(X_train, y_train)


    # Use the model
    # We'll evaluate the model's performance on train, first
    in_sample_accuracy = knn.score(X_train, y_train)
    
    out_of_sample_accuracy = knn.score(X_validate, y_validate)

    output = {
        "Neighbors": i,
        "train_accuracy": in_sample_accuracy,
        "validate_accuracy": out_of_sample_accuracy
    }
    
    metrics.append(output)
    
df = pd.DataFrame(metrics)
df["difference"] = df.train_accuracy - df.validate_accuracy
df

,Neighbors,train_accuracy,validate_accuracy,difference
0,1,0.987928,0.714953,0.272974
1,2,0.830986,0.672897,0.158089
2,3,0.847082,0.728972,0.118111
3,4,0.792757,0.682243,0.110514
4,5,0.774648,0.738318,0.036330
5,6,0.766600,0.705607,0.060992
6,7,0.764588,0.714953,0.049634
7,8,0.746479,0.710280,0.036198
8,9,0.758551,0.738318,0.020234
9,10,0.760563,0.686916,0.073647


In [22]:
df2= df.loc[df['Neighbors'] == 10].append(df.loc[df['Neighbors'] == 20]).append(df.loc[df['Neighbors'] == 1])

In [24]:
df2 = df2.sort_index().reset_index(drop=True)
df2


,Neighbors,train_accuracy,validate_accuracy,difference
0,1,0.987928,0.714953,0.272974
1,10,0.760563,0.686916,0.073647
2,20,0.718310,0.700935,0.017375
